# LAST: 0.217161
# BEST: 0.217487

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime

In [2]:
DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

In [3]:
from keras.preprocessing.text import Tokenizer

MAX_WORD_NUMS = 10000

word_tokenizer = Tokenizer(MAX_WORD_NUMS)
word_tokenizer.fit_on_texts(question_data["words"])

word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_WORD_NUMS]].values
    ),
    axis=0
)
word_embedding_data.shape

Using TensorFlow backend.


(10001, 300)

In [4]:
from keras.preprocessing.sequence import pad_sequences

WORD_SEQ_LEN = 30

def gen_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_data(train_data)
test_word1, test_word2 = gen_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape

((254386, 30), (254386, 30), (172956, 30), (172956, 30))

In [5]:
label = train_data["label"].values

In [6]:
from keras.layers import Input, Embedding, Dropout, Dense, BatchNormalization, K, LSTM
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate

DROP_RATE = 0.25
LSTM_SIZE_1 = 256
LSTM_SIZE_2 = 256
DENSE_SIZE = 300
BATCH_SIZE = 2048
NUM_EPOCHES = 50

In [7]:
# from sklearn.model_selection import train_test_split

# train_word1, dev_word1, train_word2, dev_word2, train_y, dev_y = train_test_split(
#     word1, word2, train_data["label"].values,
#     test_size=0.2
# )

# word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
# word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")

# embedding_layer = Embedding(
#     input_dim=word_embedding_data.shape[0],
#     output_dim=word_embedding_data.shape[1],
#     weights=[word_embedding_data],
#     input_length=WORD_SEQ_LEN,
#     trainable=False
# )

# word_vector1 = embedding_layer(word_input1)
# word_vector2 = embedding_layer(word_input2)

# lstm_layer1 = LSTM(LSTM_SIZE_1, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=True)
# word_first_1 = lstm_layer1(word_vector1)
# word_first_1 = Dropout(DROP_RATE)(word_first_1)
# word_first_2 = lstm_layer1(word_vector2)
# word_first_2 = Dropout(DROP_RATE)(word_first_2)

# lstm_layer2 = LSTM(LSTM_SIZE_2, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=False)
# word_second_1 = lstm_layer2(word_first_1)
# word_second_2 = lstm_layer2(word_first_2)

# x = concatenate([word_second_1, word_second_2])
# x = Dropout(DROP_RATE)(x)
# x = BatchNormalization()(x)

# x = Dense(DENSE_SIZE, activation="relu")(x)
# x = Dropout(DROP_RATE)(x)
# x = BatchNormalization()(x)

# pred = Dense(1, activation="sigmoid")(x)

# model = Model(inputs=[word_input1, word_input2], outputs=pred)
# model.compile(
#     optimizer="nadam",
#     loss="binary_crossentropy",
#     metrics=["acc"]
# )

# early_stopping = EarlyStopping("val_loss", patience=10)
# check_point = ModelCheckpoint(
#     "./log/%s.multi_lstm.{epoch:03d}.hdf5" % (datetime.now().strftime("%Y%m%d-%H%M%S")),
#     monitor="val_loss",
#     save_best_only=True,
#     save_weights_only=True
# )

# train_res = model.fit(
#     x=[train_word1, train_word2],
#     y=train_y,
#     batch_size=BATCH_SIZE,
#     epochs=NUM_EPOCHES,
#     validation_data=([dev_word1, dev_word2], dev_y),
#     shuffle=True,
#     callbacks=[early_stopping, check_point]
# )

# print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
# model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))

# test_pred = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)
# pd.DataFrame(test_pred, columns=["y_pre"]).to_csv("./result/pred.csv", index=False)

In [8]:
from sklearn.model_selection import StratifiedKFold

pred_collect = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10).split(X=word1, y=label)):
    train_word1, train_word2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]
    dev_word1, dev_word2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]

    word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
    word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")

    embedding_layer = Embedding(
        input_dim=word_embedding_data.shape[0],
        output_dim=word_embedding_data.shape[1],
        weights=[word_embedding_data],
        input_length=WORD_SEQ_LEN,
        trainable=False
    )

    word_vector1 = embedding_layer(word_input1)
    word_vector2 = embedding_layer(word_input2)

    lstm_layer1 = LSTM(LSTM_SIZE_1, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=True)
    word_first_1 = lstm_layer1(word_vector1)
    word_first_1 = Dropout(DROP_RATE)(word_first_1)
    word_first_2 = lstm_layer1(word_vector2)
    word_first_2 = Dropout(DROP_RATE)(word_first_2)

    lstm_layer2 = LSTM(LSTM_SIZE_2, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=False)
    word_second_1 = lstm_layer2(word_first_1)
    word_second_2 = lstm_layer2(word_first_2)

    x = concatenate([word_second_1, word_second_2])
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)

    x = Dense(DENSE_SIZE, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)

    pred = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=[word_input1, word_input2], outputs=pred)
    model.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )

    early_stopping = EarlyStopping("val_loss", patience=10)
    check_point = ModelCheckpoint(
        "./log/%s.multi_lstm.{epoch:03d}.hdf5" % (datetime.now().strftime("%Y%m%d-%H%M%S")),
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True
    )

    train_res = model.fit(
        x=[train_word1, train_word2],
        y=train_y,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHES,
        validation_data=([dev_word1, dev_word2], dev_y),
        shuffle=True,
        callbacks=[early_stopping, check_point]
    )

    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))

    test_pred = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)
    pred_collect.append(pd.DataFrame(test_pred, columns=["y_pre"]))

pd.DataFrame(pd.concat(pred_collect, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-pred.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),
    index=False
)

Train on 228946 samples, validate on 25440 samples
Epoch 1/50
228946/228946 [==============================] - 112s 490us/step - loss: 0.4825 - acc: 0.7704 - val_loss: 0.3700 - val_acc: 0.8417
Epoch 2/50
228946/228946 [==============================] - 109s 476us/step - loss: 0.3555 - acc: 0.8402 - val_loss: 0.3143 - val_acc: 0.8664
Epoch 3/50
228946/228946 [==============================] - 110s 481us/step - loss: 0.3103 - acc: 0.8635 - val_loss: 0.2773 - val_acc: 0.8804
Epoch 4/50
228946/228946 [==============================] - 110s 482us/step - loss: 0.2797 - acc: 0.8784 - val_loss: 0.2523 - val_acc: 0.8926
Epoch 5/50
228946/228946 [==============================] - 110s 479us/step - loss: 0.2590 - acc: 0.8886 - val_loss: 0.2428 - val_acc: 0.8953
Epoch 6/50
228946/228946 [==============================] - 110s 479us/step - loss: 0.2443 - acc: 0.8956 - val_loss: 0.2340 - val_acc: 0.8997
Epoch 7/50
228946/228946 [==============================] - 109s 478us/step - loss: 0.2306 - acc:

Epoch 18/50
228947/228947 [==============================] - 111s 485us/step - loss: 0.1726 - acc: 0.9282 - val_loss: 0.1944 - val_acc: 0.9226
Epoch 19/50
228947/228947 [==============================] - 111s 485us/step - loss: 0.1700 - acc: 0.9303 - val_loss: 0.1988 - val_acc: 0.9233
Epoch 20/50
228947/228947 [==============================] - 110s 481us/step - loss: 0.1661 - acc: 0.9308 - val_loss: 0.2007 - val_acc: 0.9227
Epoch 21/50
228947/228947 [==============================] - 111s 485us/step - loss: 0.1649 - acc: 0.9321 - val_loss: 0.1972 - val_acc: 0.9238
Epoch 22/50
228947/228947 [==============================] - 111s 484us/step - loss: 0.1635 - acc: 0.9325 - val_loss: 0.1979 - val_acc: 0.9234
Epoch 23/50
228947/228947 [==============================] - 111s 483us/step - loss: 0.1612 - acc: 0.9336 - val_loss: 0.1979 - val_acc: 0.9245
Epoch 24/50
228947/228947 [==============================] - 111s 484us/step - loss: 0.1592 - acc: 0.9344 - val_loss: 0.1986 - val_acc: 0.9241

Epoch 14/50
228948/228948 [==============================] - 110s 481us/step - loss: 0.1848 - acc: 0.9233 - val_loss: 0.2141 - val_acc: 0.9145
Epoch 15/50
228948/228948 [==============================] - 110s 481us/step - loss: 0.1815 - acc: 0.9254 - val_loss: 0.2138 - val_acc: 0.9152
Epoch 16/50
228948/228948 [==============================] - 110s 481us/step - loss: 0.1789 - acc: 0.9256 - val_loss: 0.2171 - val_acc: 0.9146
Epoch 17/50
228948/228948 [==============================] - 110s 481us/step - loss: 0.1756 - acc: 0.9271 - val_loss: 0.2140 - val_acc: 0.9165
Epoch 18/50
228948/228948 [==============================] - 110s 481us/step - loss: 0.1732 - acc: 0.9279 - val_loss: 0.2076 - val_acc: 0.9182
Epoch 19/50
228948/228948 [==============================] - 110s 480us/step - loss: 0.1712 - acc: 0.9289 - val_loss: 0.2179 - val_acc: 0.9156
Epoch 20/50
228948/228948 [==============================] - 110s 482us/step - loss: 0.1681 - acc: 0.9307 - val_loss: 0.2090 - val_acc: 0.9176

Epoch 12/50
228948/228948 [==============================] - 113s 493us/step - loss: 0.1932 - acc: 0.9196 - val_loss: 0.2101 - val_acc: 0.9160
Epoch 13/50
228948/228948 [==============================] - 113s 494us/step - loss: 0.1875 - acc: 0.9221 - val_loss: 0.2033 - val_acc: 0.9189
Epoch 14/50
228948/228948 [==============================] - 113s 493us/step - loss: 0.1838 - acc: 0.9233 - val_loss: 0.2052 - val_acc: 0.9184
Epoch 15/50
228948/228948 [==============================] - 113s 493us/step - loss: 0.1817 - acc: 0.9246 - val_loss: 0.2079 - val_acc: 0.9187
Epoch 16/50
228948/228948 [==============================] - 113s 493us/step - loss: 0.1766 - acc: 0.9264 - val_loss: 0.1978 - val_acc: 0.9206
Epoch 17/50
228948/228948 [==============================] - 113s 493us/step - loss: 0.1737 - acc: 0.9283 - val_loss: 0.2048 - val_acc: 0.9207
Epoch 18/50
228948/228948 [==============================] - 113s 493us/step - loss: 0.1707 - acc: 0.9295 - val_loss: 0.2070 - val_acc: 0.9212

Epoch 3/50
228948/228948 [==============================] - 114s 500us/step - loss: 0.3085 - acc: 0.8645 - val_loss: 0.2695 - val_acc: 0.8829
Epoch 4/50
228948/228948 [==============================] - 114s 500us/step - loss: 0.2800 - acc: 0.8786 - val_loss: 0.2571 - val_acc: 0.8907
Epoch 5/50
228948/228948 [==============================] - 114s 497us/step - loss: 0.2585 - acc: 0.8885 - val_loss: 0.2319 - val_acc: 0.9003
Epoch 6/50
228948/228948 [==============================] - 114s 497us/step - loss: 0.2444 - acc: 0.8949 - val_loss: 0.2251 - val_acc: 0.9046
Epoch 7/50
228948/228948 [==============================] - 114s 500us/step - loss: 0.2306 - acc: 0.9019 - val_loss: 0.2238 - val_acc: 0.9070
Epoch 8/50
228948/228948 [==============================] - 114s 500us/step - loss: 0.2203 - acc: 0.9071 - val_loss: 0.2144 - val_acc: 0.9120
Epoch 9/50
228948/228948 [==============================] - 114s 500us/step - loss: 0.2119 - acc: 0.9106 - val_loss: 0.2120 - val_acc: 0.9108
Epoch 